In [1]:
import numpy as np
import itertools

# input data
segment_lengths = np.array([3, 3, 2, 1, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 3, 1, 2, 
                            2, 2, 1, 4, 2, 4, 3, 2, 2, 2, 2, 4, 1], dtype=int)

# determine middle cells
# that is, the cell's input and output axes are parallel
is_middle_cell = np.zeros((64,), dtype=bool)
segment_lengths_cumsum = np.cumsum(segment_lengths)
for segment_index, segment_length in enumerate(segment_lengths):
    start_cell_index = segment_lengths_cumsum[segment_index - 1] if segment_index > 0 else 0
    end_cell_index = start_cell_index + segment_length - 1
    if start_cell_index == end_cell_index:
        # case 1: the segment length is 1
        is_middle_cell[start_cell_index] = True
    else:
        # case 2: the segment length is n > 1, and the cell is not the 0th or the (n-1)th
        for i in range(start_cell_index + 1, end_cell_index):
            is_middle_cell[i] = True

In [2]:
possible_axes = ['+x', '-x', '+y', '-y', '+z', '-z']

next_location_delta_table = {
    '+x': (1, 0, 0),
    '-x': (-1, 0, 0),
    '+y': (0, 1, 0),
    '-y': (0, -1, 0),
    '+z': (0, 0, 1),
    '-z': (0, 0, -1)
}    

def get_location(prev_location, prev_axis, filled_cells):
    x, y, z = prev_location
    dx, dy, dz = next_location_delta_table[prev_axis]
    location = x + dx, y + dy, z + dz
    for val in location:
        if val >= 4 or val < 0:
            return None
    if filled_cells[location] == True:
        return None
    return location

def get_axes(cell_index, prev_axis):
    assert prev_axis in possible_axes
    if is_middle_cell[cell_index]:
        return [prev_axis]
    if prev_axis in {'+x', '-x'}:
        return ['+y', '-y', '+z', '-z']
    if prev_axis in {'+y', '-y'}:
        return ['+x', '-x', '+z', '-z']
    if prev_axis in {'+z', '-z'}:
        return ['+x', '-x', '+y', '-y']

In [4]:
def solve(index, filled_cells, cell_locations, cell_axes):
    # base case
    if index == 0:
        for init_location in itertools.product(*([list(range(2))] * 3)):
            print("init location", init_location)
            for init_axis in possible_axes:
                # fill the 0-th cell and axis
                cell_locations[0] = init_location
                cell_axes[0] = init_axis
                filled_cells[cell_locations[0]] = True
                # solve
                res = solve(1, filled_cells, cell_locations, cell_axes)
                if res is not None:
                    return res
                # clean up
                filled_cells[cell_locations[0]] = False
                cell_locations[0] = None
                cell_axes[0] = None
        return None
    
    # get location
    location = get_location(cell_locations[index - 1],
                            cell_axes[index - 1],
                            filled_cells)
    if location is None:
        return None
    
    # fill
    cell_locations[index] = location
    filled_cells[location] = True
    
    # done if index == 63, cell_locations, cell_axes fully filled
    if index == 63:
        return (cell_locations, cell_axes)
    
    # get axes
    axes = get_axes(index, cell_axes[index - 1])
    for axis in axes:
        # fill axis
        cell_axes[index] = axis
        # try solving recursively
        res = solve(index + 1, filled_cells, cell_locations, cell_axes)
        if res is not None:
            return res
        # clean up
        cell_axes[index] = None
    
    # not successful, clean up
    cell_locations[index] = None
    filled_cells[location] = False
    
    return None

def solve_with_init():
    # map of the filled cells
    filled_cells = np.zeros((4, 4, 4), dtype=bool)

    # cells[i] = [x, y, z], location for the i-th cell
    cell_locations = [None] * 64

    # axes[i] is the vector direction from cells[i] to cells[i+1]
    cell_axes = [None] * 63
    
    return solve(0, filled_cells, cell_locations, cell_axes)

print(solve_with_init())

(0, 0, 0)
solving max_index = 1
solving max_index = 2
solving max_index = 3
solving max_index = 4
solving max_index = 5
solving max_index = 6
solving max_index = 7
solving max_index = 8
solving max_index = 9
solving max_index = 10
solving max_index = 11
solving max_index = 12
solving max_index = 13
solving max_index = 14
solving max_index = 15
solving max_index = 16
solving max_index = 17
solving max_index = 18
solving max_index = 19
solving max_index = 20
solving max_index = 21
solving max_index = 22
solving max_index = 23
solving max_index = 24
solving max_index = 25
solving max_index = 26
solving max_index = 27
solving max_index = 28
solving max_index = 29
solving max_index = 30
solving max_index = 31
solving max_index = 32
solving max_index = 33
solving max_index = 34
solving max_index = 35
solving max_index = 36
solving max_index = 37
solving max_index = 38
solving max_index = 39
solving max_index = 40
solving max_index = 41
solving max_index = 42
solving max_index = 43
solving ma

IndexError: list assignment index out of range